In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import display
from pathlib import Path

base = '../datasets/RepCount/'

In [18]:
base = '../datasets/RepCount'
train_csv = os.path.join(base, 'annotation/train.csv')
test_csv = os.path.join(base, 'annotation/test.csv')
val_csv = os.path.join(base, 'annotation/val.csv')

train_df = pd.read_csv(train_csv, index_col=0)
test_df = pd.read_csv(test_csv, index_col=0)
val_df = pd.read_csv(val_csv, index_col=0)

print(len(train_df), len(test_df), len(val_df))
print('total:', len(train_df) + len(test_df) + len(val_df))


train_df.dropna(subset=['count'], inplace=True)
train_df.head()


758 152 131
total: 1041


,type,name,count,L1,L2,L3,L4,L5,L6,L7,...,L293,L294,L295,L296,L297,L298,L299,L300,L301,L302
0,frontraise,train951.mp4,4.0,6.0,72.0,72.0,132.0,132.0,204.0,204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,frontraise,train952.mp4,10.0,13.0,62.0,62.0,103.0,103.0,126.0,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pullups,test1463.mp4,7.0,21.0,60.0,60.0,98.0,98.0,132.0,132.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,squant,test2340.mp4,4.0,0.0,86.0,86.0,155.0,155.0,218.0,219.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,front_raise,stu5_11.mp4,6.0,104.0,179.0,179.0,255.0,255.0,330.0,426.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## CLEAN

I am dying cleaning this dataset.
Hope there is no mislabeled data in the world and everyone gets beautifully formatted data.😭😭😭😖😖😖

In [19]:
# Labeld frame is longer than actual frame length. I don't know why. 
# It does has 1841 frames. But only 1681 frames in rawframe dir.
# I know why. The video is broken.
train_df[train_df['name'] == 'stu1_10.mp4']
train_df = train_df[train_df['name'] != 'stu1_10.mp4']

In [20]:
train_df[train_df['name'] == 'stu6_5.mp4'].values[0][3:28]
train_df = train_df[train_df['name'] != 'stu6_5.mp4']
train_df.drop(train_df[train_df['name'] == 'stu4_3'].index, inplace=True)

In [21]:
test_df[test_df['name'] == 'stu4_5.mp4'].values[0]
# why are there so many same frames in the repetition?
# And the video is situp, not bench press.
# Drop this.
test_df = test_df[test_df['name'] != 'stu4_5.mp4']

In [22]:
def get_reps(df, name):
    reps = df[df['name'] == name].values[0][3:]
    reps = [r for r in reps if pd.notnull(r)]
    count = df[df['name'] == name].values[0][2]
    return reps

reps = get_reps(test_df, 'train1615.mp4')
print(len(reps))
print(reps)

20
[0.0, 25.0, 27.0, 57.0, 58.0, 85.0, 87.0, 117.0, 118.0, 148.0, 149.0, 179.0, 180.0, 208.0, 208.0, 239.0, 239.0, 267.0, 268.0, 297.0]


## Original YouTube data

In [23]:
original_data = Path(base, 'original_data/filename_mapping.xlsx')
ori = pd.read_excel(original_data, index_col=0)
print(len(ori))
ori[ori['vid']=='REjznoKN8Q8'].head(10)

800


,class,vid_st,vid,stuid,rename
693,squat,REjznoKN8Q8_143.mp4,REjznoKN8Q8,8,stu8_69
694,squat,REjznoKN8Q8_188.mp4,REjznoKN8Q8,3,stu3_67
779,squat,REjznoKN8Q8_188.mp4,REjznoKN8Q8,8,stu8_75


In [24]:
def min2sec(t):
    m, s = t.split(':')
    return int(m)*60 + int(s)

d_ = dict()
download_link_dir = Path(base, 'original_data/DownloadLink')
for fname in os.listdir(download_link_dir):
    with open(Path(download_link_dir, fname), 'r') as f:
        lines = f.readlines()
        for line in lines:
            vid, st, et = line.strip().split()
            st = min2sec(st)
            et = min2sec(et)
            vid_st = f'{vid}_{st}.mp4'
            d_[vid_st] = et

for row in ori.itertuples():
    vid_st = row.vid_st
    if vid_st in d_:
        ori.at[row.Index, 'et'] = int(d_[vid_st])
        ori.at[row.Index, 'st'] = int(vid_st[12:-4])

In [25]:
# add column 'name' to ori
ori['name'] = ori['rename'].apply(lambda x: x+'.mp4')
ori.drop(columns=['class'], inplace=True)
ori[ori['et'].notna()].head()


,vid_st,vid,stuid,rename,et,st,name
0,1pzpaYrhdOo_13.mp4,1pzpaYrhdOo,1,stu1_0,22.0,13.0,stu1_0.mp4
1,1pzpaYrhdOo_93.mp4,1pzpaYrhdOo,4,stu4_0,105.0,93.0,stu4_0.mp4
2,7wDx6mZDxA8_0.mp4,7wDx6mZDxA8,3,stu3_0,12.0,0.0,stu3_0.mp4
3,DcRj7jtKhk4_52.mp4,DcRj7jtKhk4,10,stu10_0,62.0,52.0,stu10_0.mp4
4,FyGBSs1X1qM_105.mp4,FyGBSs1X1qM,3,stu3_1,114.0,105.0,stu3_1.mp4


In [26]:
# reps to one column

train_df['reps'] = train_df.apply(lambda x: ' '.join([str(int(y)) for y in x.values[3:] if pd.notna(y)]), axis=1)
val_df['reps'] = val_df.apply(lambda x: ' '.join([str(int(y)) for y in x.values[3:] if pd.notna(y)]), axis=1)
test_df['reps'] = test_df.apply(lambda x: ' '.join([str(int(y)) for y in x.values[3:] if pd.notna(y)]), axis=1)

train_df.drop(train_df.columns[3:-1], axis=1, inplace=True)
val_df.drop(val_df.columns[3:-1], axis=1, inplace=True)
test_df.drop(test_df.columns[3:-1], axis=1, inplace=True)

# train_df.apply(lambda x: print(int(x['count']), x['reps']), axis=1)

# train_df['reps'] = train_df.apply(lambda x: ' '.join([str(int(y)) for y in x['reps'] if y>=0]), axis=1)
# val_df['reps'] = val_df.apply(lambda x: ' '.join([str(int(y)) for y in x['reps'] if y>=0]), axis=1)
# test_df['reps'] = test_df.apply(lambda x: ' '.join([str(int(y)) for y in x['reps'] if y>=0 ]), axis=1)

train_df.at[train_df.index[1], 'reps']

'13 62 62 103 103 126 126 146 146 171 171 194 194 215 215 239 239 260 260 281'

In [27]:
# add YouTube ID and start time to annotation
def func(name):
    name = name.split('.')[0]
    row = ori[ori['rename'] == name]
    if len(row) == 0 or any(row['vid'].isna()):
        return None
    vid_st = row['vid_st'].values[0]

    vid = vid_st[:11]
    st = vid_st[12:-4]
    return vid, int(st)

train_df = train_df.merge(ori, how='left', on='name', suffixes=[None, None])
val_df = val_df.merge(ori, how='left', on='name', suffixes=[None, None])
test_df = test_df.merge(ori, how='left', on='name', suffixes=[None, None])

In [28]:
print(len(train_df[train_df['vid'].notna()]), len(val_df[val_df['vid'].notna()]), len(test_df[test_df['vid'].notna()]))


506 91 108


## Fixing the data
Many different names for the same thing.
Fixing it.

In [29]:
pushups = train_df[train_df['type'] == 'pushups']
pullups = train_df[train_df['type'] == 'pullups']
push_up = train_df[train_df['type'] == 'push_up']
pull_up = train_df[train_df['type'] == 'pull_up']

# replace column pushups to push_up, pullups to pull_up, squant to squat, jumpjacks to jump_jack
train_df['type'] = train_df['type'].replace(
    ['pushups', 'pullups', 'squant', 'jumpjacks', 'benchpressing', 'frontraise'],
    ['push_up', 'pull_up', 'squat', 'jump_jack', 'bench_pressing', 'front_raise'])

# val and test too
val_df['type'] = val_df['type'].replace(
    ['pushups', 'pullups', 'squant', 'jumpjacks', 'benchpressing', 'frontraise'],
    ['push_up', 'pull_up', 'squat', 'jump_jack', 'bench_pressing', 'front_raise'])

test_df['type'] = test_df['type'].replace(
    ['pushups', 'pullups', 'squant', 'jumpjacks', 'benchpressing', 'frontraise'],
    ['push_up', 'pull_up', 'squat', 'jump_jack', 'bench_pressing', 'front_raise'])


In [30]:
classes = train_df['type'].unique()
df1 = train_df.groupby('type').count()
df1 = df1[['count']].rename(columns={'count': 'count_train'})

df2 = val_df.groupby('type').count()
df2 = df2[['count']].rename(columns={'count': 'count_val'})

df3 = test_df.groupby('type').count()
df3 = df3[['count']].rename(columns={'count': 'count_test'})

# show df1 df2 df3 in one table
df = pd.concat([df1, df2, df3], axis=1)
df.index.name = 'type'
df.fillna(0, inplace=True)
df = df.astype(int)
df = df.reset_index()
df.sort_values('count_train', ascending=False)

,type,count_train,count_val,count_test
9,squat,101,16,18
6,pull_up,94,14,19
2,front_raise,93,19,18
8,situp,93,18,20
1,bench_pressing,91,13,18
7,push_up,88,18,16
3,jump_jack,76,15,26
5,pommelhorse,69,15,15
4,others,37,0,1
0,battle_rope,13,3,0


### Save all to new csv

Remove ['battle_rope', 'others', 'pommelhorse']

In [31]:
train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'
all_df = pd.concat([train_df, val_df, test_df], axis=0)

# drop types battle_rope, others, pommelhorse
to_drop = all_df[all_df['type'].isin(['battle_rope', 'others', 'pommelhorse'])].index
all_df.drop(to_drop, inplace=True)
all_df.rename(columns={'type': 'class_', 'st': 'start', 'et': 'end'}, inplace=True)
# reorder columns
keep_cols = ['class_', 'split', 'name', 'vid', 'start', 'end', 'count', 'reps']

all_df = all_df[keep_cols]

all_df[all_df['split']=='test']

,class_,split,name,vid,start,end,count,reps
0,situp,test,train3344.mp4,NaN,NaN,NaN,14.0,1 23 23 40 40 58 59 75 75 92 92 109 109 126 12...
1,push_up,test,stu8_42.mp4,BVuI-syjR6A,0.0,15.0,6.0,0 42 42 88 88 142 142 199 199 253 253 317
2,pull_up,test,stu2_39.mp4,BU-X16wG-Jo,138.0,156.0,6.0,41 109 109 190 190 258 258 336 336 396 396 461
3,bench_pressing,test,train151.mp4,NaN,NaN,NaN,3.0,13 33 33 51 51 74
4,jump_jack,test,train1615.mp4,NaN,NaN,NaN,10.0,0 25 27 57 58 85 87 117 118 148 149 179 180 20...
...,...,...,...,...,...,...,...,...
131,situp,test,stu8_57.mp4,zoL7eingzkQ,40.0,53.0,7.0,2 53 53 106 106 160 160 219 219 274 274 335 33...
132,push_up,test,stu8_46.mp4,wS6Kp3k-zV8,70.0,112.0,19.0,37 82 82 132 132 196 196 244 244 295 295 340 3...
133,push_up,test,stu4_47.mp4,t0QZmiXBC-Q,62.0,120.0,37.0,84 104 105 132 132 156 156 187 188 214 215 241...
134,bench_pressing,test,stu2_6.mp4,QxLPG9FwAUU,90.0,123.0,4.0,121 438 440 535 537 616 617 680


In [32]:
all_df.to_csv(Path(base, 'all_data.csv'))